## First we create the deforestation "view"

In [3]:
import pandas as pd

bdforest = pd.read_csv('./desktop/python/forest_area.csv')
bdland = pd.read_csv('./desktop/python/land_area.csv')
bdregions = pd.read_csv('./desktop/python/regions.csv')
bdregions.head()

,country_name,country_code,region,income_group
0,Afghanistan,AFG,South Asia,Low income
1,Albania,ALB,Europe & Central Asia,Upper middle income
2,Algeria,DZA,Middle East & North Africa,Upper middle income
3,American Samoa,ASM,East Asia & Pacific,Upper middle income
4,Andorra,AND,Europe & Central Asia,High income


In [5]:
bdforest.head()

,country_code,country_name,year,forest_area_sqkm
0,ABW,Aruba,2016,4.20000
1,AFG,Afghanistan,2016,13500.00000
2,AGO,Angola,2016,577311.99220
3,ALB,Albania,2016,7705.39978
4,AND,Andorra,2016,160.00000


In [4]:
bdland = pd.read_csv('./desktop/python/land_area.csv')

In [6]:
bdland.head()

,country_code,country_name,year,total_area_sq_mi
0,ABW,Aruba,2016,69.50
1,AFG,Afghanistan,2016,252069.50
2,AGO,Angola,2016,481351.35
3,ALB,Albania,2016,10579.15
4,AND,Andorra,2016,181.47


### Change total area sq miles to sq km in the land area db

In [14]:
### Insert column sq km Change total area sq miles to sq km
## Example
##         df['salary_rounded'] = round(df['salary'] * 0.85, 2)

bdland['total_area_sqkm'] = round(bdland['total_area_sq_mi'] * 2.59,2)

## Erase area sq miles field
## Example
##         df = df.drop('salary', axis=1) axis=1 refers to columns, whereas axis = 0 is rows

bdland = bdland.drop('total_area_sq_mi', axis=1)
bdland.head()

,country_code,country_name,year,total_area_sqkm
0,ABW,Aruba,2016,180.00
1,AFG,Afghanistan,2016,652860.00
2,AGO,Angola,2016,1246700.00
3,ALB,Albania,2016,27400.00
4,AND,Andorra,2016,470.01


### Merge tables so we would simulate the deforestation view

In [68]:
# perform the inner joins on country_code and year columns
deforestation = pd.merge(bdforest, bdland, on=['country_code', 'year'], how='inner')
deforestation = pd.merge(deforestation, bdregions, on='country_code', how='inner')

# filter out rows where forest_area_sqkm is null
deforestation = deforestation[deforestation['forest_area_sqkm'].notnull()]

# calculate the forest area as a percentage of total area in square miles
# Here we are using the new field total_area_sqkm instead of the miles one
deforestation['frst_percent'] = deforestation['forest_area_sqkm'] * 100 / (deforestation['total_area_sqkm'])

deforestation.head()

,country_code,country_name_x,year,forest_area_sqkm,country_name_y,total_area_sqkm,country_name,region,income_group,frst_percent
0,ABW,Aruba,2016,4.2,Aruba,180.0,Aruba,Latin America & Caribbean,High income,2.333333
1,ABW,Aruba,2015,4.2,Aruba,180.0,Aruba,Latin America & Caribbean,High income,2.333333
2,ABW,Aruba,2014,4.2,Aruba,180.0,Aruba,Latin America & Caribbean,High income,2.333333
3,ABW,Aruba,2013,4.2,Aruba,180.0,Aruba,Latin America & Caribbean,High income,2.333333
4,ABW,Aruba,2012,4.2,Aruba,180.0,Aruba,Latin America & Caribbean,High income,2.333333


### We have 2 fields with the same name, we drop one and rename the other, 
### also we will create the percentage of forest area column

### In order to run this part, we have to re run the former

In [69]:
# We rename the remaining column
deforestation.rename(columns={'country_name_x': 'country_name'})

# First we drop the redundant column, don't forget the inplace so drop would be definitive
deforestation.drop('country_name_y', axis=1,  inplace=True)
deforestation.drop('country_name_x', axis=1,  inplace=True)


print(deforestation.columns)
deforestation.head()



Index(['country_code', 'year', 'forest_area_sqkm', 'total_area_sqkm',
       'country_name', 'region', 'income_group', 'frst_percent'],
      dtype='object')


,country_code,year,forest_area_sqkm,total_area_sqkm,country_name,region,income_group,frst_percent
0,ABW,2016,4.2,180.0,Aruba,Latin America & Caribbean,High income,2.333333
1,ABW,2015,4.2,180.0,Aruba,Latin America & Caribbean,High income,2.333333
2,ABW,2014,4.2,180.0,Aruba,Latin America & Caribbean,High income,2.333333
3,ABW,2013,4.2,180.0,Aruba,Latin America & Caribbean,High income,2.333333
4,ABW,2012,4.2,180.0,Aruba,Latin America & Caribbean,High income,2.333333


In [70]:
deforestation.tail()

,country_code,year,forest_area_sqkm,total_area_sqkm,country_name,region,income_group,frst_percent
5881,ZWE,1994,208560.0,386849.99,Zimbabwe,Sub-Saharan Africa,Low income,53.912371
5882,ZWE,1993,211830.0,386849.99,Zimbabwe,Sub-Saharan Africa,Low income,54.757659
5883,ZWE,1992,215100.0,386849.99,Zimbabwe,Sub-Saharan Africa,Low income,55.602948
5884,ZWE,1991,218370.0,386849.99,Zimbabwe,Sub-Saharan Africa,Low income,56.448237
5885,ZWE,1990,221640.0,386849.99,Zimbabwe,Sub-Saharan Africa,Low income,57.293526


In [130]:
## Calculate total area of forest in the world in 2016
total_forest_area_2016 = deforestation[(deforestation['region'] == 'World')]
total_forest_area_2016 = total_forest_area_2016[(total_forest_area_2016['year'] == 2016)]
total_forest_area_2016 = total_forest_area_2016['forest_area_sqkm'].sum()


## Calculate total area of forest in the world in 1990
total_forest_area_1990 = deforestation[(deforestation['region'] == 'World')]
total_forest_area_1990 = total_forest_area_1990[(total_forest_area_1990['year'] == 1990)]
total_forest_area_1990 = total_forest_area_1990['forest_area_sqkm'].sum()

pd.options.display.float_format = '{:,.0f}'.format
print(f"Total forest area in 1990 = {total_forest_area_1990}\nTotal forest area in 2016 = {total_forest_area_2016}\nPercentage change in forest area = {round((total_forest_area_1990-total_forest_area_2016)/total_forest_area_1990,3)}")

Total forest area in 1990 = 41282694.9
Total forest area in 2016 = 39958245.9
Percentage change in forest area = 0.032


In [146]:
temp2016 = deforestation[deforestation['year'] == 2016].groupby(['region', 'year'])['frst_percent'].avg()

temp1990 = deforestation[deforestation['year'] == 1990].groupby(['region', 'year'])['frst_percent'].avg()


AttributeError: 'SeriesGroupBy' object has no attribute 'avg'

In [145]:
compare = pd.merge(temp2016, temp1990, on=['region'], how='inner')
compare.rename(columns={'forest_area_sqkm_x': 'forest area 2016'},inplace=True)
compare.rename(columns={'forest_area_sqkm_y': 'forest area 1990'},inplace=True)
compare['difference'] = compare['forest area 2016']-compare['forest area 1990']
compare

,forest area 2016,forest area 1990,difference
region,,,
East Asia & Pacific,"6,421,326","6,280,253","141,074"
Europe & Central Asia,"10,438,609","10,199,848","238,762"
Latin America & Caribbean,"9,250,586","10,242,342","-991,756"
Middle East & North Africa,"232,131","199,293","32,838"
North America,"6,573,934","6,507,240","66,694"
South Asia,"835,310","789,187","46,123"
Sub-Saharan Africa,"6,115,291","6,515,615","-400,324"
World,"39,958,246","41,282,695","-1,324,449"
